In [1]:
import os, sys, logging

if "../" not in sys.path:
    sys.path.append("../")

from data_generator import *
from feature import FeatureGenerator, FeatureConfig
from util.hedge_log import initlog
import numpy as np
import queue
import joblib


begin_time = datetime.datetime(2024, 3, 27, 21,tzinfo=TZ_8)
end_time = datetime.datetime(2024, 3, 27, 21, 59,tzinfo=TZ_8)
exchange = 'binance'
symbol = 'btc_usdt'

trade_prc = []

for idx, row in enumerate(
    get_data_generator(begin_time, end_time, exchange, symbol)
):

    if row[1] == "trade":
        """
            ({'aggressor_side': 'sell', 'price': 42590.13, 'size': 0.00117, 'ts_event': 1702746000004000000}, 'trade')
        """
        trade_prc.append([row[0].ts_event, row[0].price])


fe_list = joblib.load('fe_list.pkl')

tmp = np.array(trade_prc)
fe_list = np.array(fe_list)

26.731910228729248
26.796015977859497
26.796046495437622


In [14]:
fe_list[:4]

array([[['Breadth_2_4', None, None],
        ['Immediacy_2_4', None, None],
        ['VolumeAvg_2_4', None, None]],

       [['Breadth_2_4', None, None],
        ['Immediacy_2_4', None, None],
        ['VolumeAvg_2_4', None, None]],

       [['Breadth_2_4', None, None],
        ['Immediacy_2_4', None, None],
        ['VolumeAvg_2_4', None, None]],

       [['Breadth_2_4', None, None],
        ['Immediacy_2_4', None, None],
        ['VolumeAvg_2_4', None, None]]], dtype=object)

In [17]:
fe_plot = list()
feature_num = 2
for i in range(fe_list.shape[0]):
    fe_value = fe_list[i,feature_num,:]
    if fe_value[2] is not None:
        fe_plot.append([int(fe_value[1]),float(fe_value[2])])

fe_plot = np.array(fe_plot)
if fe_plot[0][0] > 1002746002117000000000:
    fe_plot[:,0] = fe_plot[:,0]/1000

In [18]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 创建一个带有两个Y轴的图形
fig = make_subplots(specs=[[{"secondary_y": True}]])


x_time = pd.to_datetime(pd.Series(fe_plot[:,0]))

# 添加第一个数据系列到图形，使用默认的Y轴
fig.add_trace(
    go.Scatter(x= pd.to_datetime(pd.Series(tmp[:,0])), y=tmp[:,1], name="系列 1"),
    secondary_y=False,
)

# 添加第二个数据系列到图形，使用第二个Y轴
fig.add_trace(
    go.Scatter(x= x_time, y=fe_plot[:,1].reshape(-1), name="系列 2"),
    secondary_y=True,
)

# 设置图表的标题
fig.update_layout(title_text= fe_list[:,:,0][0][0])

# 设置x轴标题
fig.update_xaxes(title_text="x轴")

# 设置左侧Y轴的标题
fig.update_yaxes(title_text="主Y轴", secondary_y=False)

# 设置右侧Y轴的标题
fig.update_yaxes(title_text="次Y轴", secondary_y=True)

fig.write_html("feature_plot/f_plot_"+fe_list[:,:,0][0][feature_num]+".html")